In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score
from sklearn.preprocessing import RobustScaler
import random

In [2]:
def initialize_population(num_individuals, num_features):
    print("initialize_population method has started")
    population=list()
    for i in range(num_individuals):
        k=random.randint(1, num_features)
        population.append(random.sample(range(num_features), k))
    print("initialize_population method has ended")
    return population

In [3]:
def fitness_function(features, X_train, y_train, X_val, y_val, generation, num_generations):
    print("fitness_function method has started")
    if len(features)==0:
        print("No features were selected. Thus, returning fitness=0")
        return 0

    try:
        print(f"Features selected for computing the fitness: {features}")
        X_train_selected=X_train[:, features]
        X_val_selected=X_val[:, features]
        model=KNeighborsClassifier()
        model.fit(X_train_selected, y_train)
        y_pred=model.predict(X_val_selected)
        recall=recall_score(y_val, y_pred, average='macro')
        '''
        As per research paper:  "Data feature selection based on Artificial Bee Colony algorithm", for maximization problem we can use objective
        function value as the fitness function value.
        '''

        '''
        As we move ahead in number of generations, we assign more penalty to solutions with relatively higher number of features among the subset of
        features.
        For example: - Given the total number of features in the dataset = 45
        1) If generation number is 2, number of features is 10, then Penalty = 0.088
        2) If generation number is 10, number of features is 10, then Penalty = 0.44
        3) If generation number is 10, number of features is 20, then Penalty = 0.88
        '''
        penalty=(len(features)/X_train.shape[1])*(generation/num_generations)*2
        adjusted_recall=recall-penalty
        print("generation: ",generation)
        print("num_generations: ",num_generations)
        print("len(features): ",len(features))
        print("X_train.shape[1]",X_train.shape[1])
        #Final fitness value
        adjusted_recall=recall-penalty
        print(f"Recall for selected features: {recall}, Adjusted recall for selected features: {adjusted_recall}")
        print("fitness_function method has ended")
        return adjusted_recall
    except Exception as e:
        print(f"Error in fitness_function with features {features}: {e}")
        print("fitness_function method has ended")
        return 0

In [4]:
def employed_bee_phase(population, fitness_values, X_train, y_train, X_val, y_val, generation, num_generations):
    print("Employed bee phase has started")
    #Creating a new list that will store all individuals and their fitness values from Employed bee phase
    new_population=list()
    new_fitness_values=list()
    Np=len(population) #Number of subset of features (Number of food sources).
    global trial_counters
    print("Trial counters at start of generation ",generation,": ",trial_counters)
    for i in range(0, Np):
        individual=population[i]
        f_current=fitness_values[i]
        print("At position i =",i," Individual =",individual," and its fitness = ",f_current)
        #Generating a new solution
        print("Generating a new solution for the above individual.")
        if len(individual)>0:
            k=random.randint(0, len(individual)-1)
            print("Random value k between 0 and ",len(individual)-1," = ",k)
            new_individual=list(individual)
            available_features=list(set(range(len(individual)))-set(new_individual))
            if available_features:
                new_individual[k]=random.choice(available_features)
            print("New individual is: ",new_individual)
            new_fitness=fitness_function(new_individual, X_train, y_train, X_val, y_val, generation, num_generations)
            print("Fitness value of the new individual is: ",new_fitness)
            if(new_fitness>f_current):
                new_population.append(new_individual)
                trial_counters[i]=0
                new_fitness_values.append(new_fitness)
            else:
                new_population.append(individual)
                trial_counters[i]=trial_counters[i]+1
                new_fitness_values.append(f_current)
    print("Employee bee phase has ended")
    if len(new_population)==0:
        new_population=population
    print("In generation number: ",generation,", Population after Employed Bee phase: ",new_population)
    print("Respective fitness values: ",new_fitness_values)
    print("Trial counters at end of generation ",generation,": ",trial_counters)
    return new_population, new_fitness_values

In [5]:
def has_multiple_maximums(trials):
    max_value=max(trials)
    max_count=0
    for i in trials:
        if i==max_value:
            max_count=max_count+1
    return max_count

In [6]:
def fetching_index_for_identical_values(trials):
    pos=0
    index_list=list()
    max_value=max(trials)
    for i in trials:
        if i==max_value:
            index_list.append(pos)
            pos=pos+1
        else:
          pos=pos+1
    return index_list

In [7]:
def scout_bee_phase(population, fitness_values, X_train, y_train, X_val, y_val, generation, num_generations, limit):
    print("Scout bee phase has started")
    #Creating a new list that will store all individuals and their fitness values from Scout bee phase
    new_population=population.copy()
    new_fitness_values=fitness_values.copy()
    num_features=X_train.shape[1]
    Np=len(population) #Number of subset of features (Number of food sources).
    global trial_counters
    trial_greater_than_limit=list()
    for i in range(0, Np):
        if trial_counters[i]>limit:
            trial_greater_than_limit.append(trial_counters[i])

    print("Among the trial_counters: ",trial_counters,", following are greater than limit=5: ",trial_greater_than_limit)

    if len(trial_greater_than_limit)>1:
        #We have multiple food sources having trial counter greater than limit.
        #Case 1: All food sources do not have identical trial counter
        if has_multiple_maximums(trial_greater_than_limit)==1:
            print("We have multiple food sources with trial_counters>limit, and one of them has maximum value")
            max_trial=max(trial_greater_than_limit)
            print("Food source with trial counter: ",max_trial," has maximum value and is greater than limit.")
            max_trial_index=trial_counters.index(max_trial)
            print("Food source with trial counter: ",max_trial," lies at index: ",max_trial_index)
            #Generating a new random solution
            k=random.randint(1, num_features)
            new_random_individual=random.sample(range(num_features), k)
            new_population[max_trial_index]=new_random_individual
            new_fitness=fitness_function(new_random_individual, X_train, y_train, X_val, y_val, generation, num_generations)
            new_fitness_values[max_trial_index]=new_fitness
            #Resetting the value of trial counter
            trial_counters[max_trial_index]=0
        #Case 2: All food sources have identical trial counter
        elif has_multiple_maximums(trial_greater_than_limit)>1:
            print("We have multiple food sources with trial_counters>limit, and more than one have identical maximum values")
            max_trial=max(trial_greater_than_limit)
            print("Food source with trial counter: ",max_trial," has maximum value and is greater than limit.")
            identical_index_list=fetching_index_for_identical_values(trial_counters)
            print("Following index position in trial counter have maximum value which are also identical values: ",identical_index_list)
            #Selecting one of the above index positions randomly
            random_position=random.choice(identical_index_list)
            print("The new solution will be added at the randomly selected index: ", random_position)
            #Generating a new random solution
            k=random.randint(1, num_features)
            new_random_individual=random.sample(range(num_features), k)
            new_population[random_position]=new_random_individual
            new_fitness=fitness_function(new_random_individual, X_train, y_train, X_val, y_val, generation, num_generations)
            new_fitness_values[random_position]=new_fitness
            #Resetting the value of trial counter
            trial_counters[random_position]=0
    elif len(trial_greater_than_limit)==1:
        print("Among all food sources, only one food source has trial counter greater than limit.")
        max_trial=trial_greater_than_limit[0]
        max_trial_index=trial_counters.index(max_trial)
        #print("Maximum value of trial counter is: ",trial_counters[max_trial]," at position: ",max_trial)
        #max_trial_index=trial_counters.index(max_trial)
        print("Food source with trial counter: ",max_trial," lies at index: ",max_trial_index)
        #Generating a new random solution
        k=random.randint(1, num_features)
        new_random_individual=random.sample(range(num_features), k)
        new_fitness=fitness_function(new_random_individual, X_train, y_train, X_val, y_val, generation, num_generations)
        new_population[max_trial_index]=new_random_individual
        new_fitness_values[max_trial_index]=new_fitness
        #Resetting the value of trial counter
        trial_counters[max_trial_index]=0
    else:
        print("All food sources have trial counter lesser than limit.")

    print("Scout bee phase has ended")
    print("Trial counter after scout bee phase: ",trial_counters)
    return new_population, new_fitness_values

In [8]:
def onlooker_bee_phase(population, fitness_values, X_train, y_train, X_val, y_val, generation, num_generations):
    print("Onlooker bee phase has started")
    #Creating a new list that will store all individuals and their fitness values from Onlooker bee phase
    new_population=list()
    new_fitness_values=list()
    Np=len(population) #Number of subset of features (Number of food sources).
    global trial_counters
    #For Onlooker bee phase, we first need to check that a condition is fulfilled by each food source by computing its probability and comparing it with
    #a random number. Food sources having probability greater than the random number are passed in Onlooker bee phase.
    prob=list() #List to store probability values of the current population.
    for i in range(0, Np):
        fit_i=fitness_values[i]
        max_fitness=max(fitness_values)
        prob_i=(0.9*(fit_i/max_fitness))+0.1
        print("Probability of food source i= ",i," is ",prob_i)
        individual=population[i]
        print("Individual at position i = ",i," is ",individual)
        #Generating a random number: r
        r=random.random()
        print("Random number r = ",r)
        if prob_i>r:
            print("Probability of food source i = ",i," is greater than the random number generated for it")
            #Generating a new solution
            print("Generating a new solution for the above individual")
            if len(individual)>0:
                k=random.randint(0, len(individual)-1)
                print("Random value k between 0 and ",len(individual)-1," = ",k)
                new_individual=list(individual)
                available_features=list(set(range(len(individual)))-set(new_individual))
                if available_features:
                    new_individual[k]=random.choice(available_features)
                print("New individual is: ",new_individual)
                new_fitness=fitness_function(new_individual, X_train, y_train, X_val, y_val, generation, num_generations)
                print("Fitness value of the new individual is: ",new_fitness)
                if(new_fitness>fit_i):
                    print("Fitness of the new individual > Fitness of current individual")
                    new_population.append(new_individual)
                    trial_counters[i]=0
                    new_fitness_values.append(new_fitness)
                else:
                    print("Fitness of the new individual < Fitness of current individual")
                    new_population.append(individual)
                    trial_counters[i]=trial_counters[i]+1
                    new_fitness_values.append(new_fitness)

        else:
            print("Probability of food source i = ",i," is smaller than the random number generated for it")
            new_population.append(individual)
            new_fitness_values.append(fit_i)
    print("Onlooker bee phase has ended")
    if len(new_population)==0:
        new_population=0
    print("In generation number: ",generation,", Population after Onlooker Bee phase: ",new_population)
    print("Respective fitness values: ",new_fitness_values)
    print("Trial counters at end of generation ",generation,": ",trial_counters)
    return new_population, new_fitness_values

In [9]:
def abc_feature_selection(X, y):
    print("abc_feature_selection method has started")
    #Setting parameters for ABC algorithm
    num_features=X.shape[1] # Total number of independent features on the dataset
    num_generations=5 # Number of generations
    num_individuals=int(num_features/2) # Number of subsets of features
    limit=5 # Threshold count for an individual to fail to become eligible for Scout phase

    print("num_features: ", num_features)
    print("num_generations: ", num_generations)
    print("num_individuals: ", num_individuals)
    print("limit: ", limit)

    #Getting the initial population
    population=initialize_population(num_individuals, num_features)
    print("Initial population: \n",population)

    #Initializing the trial counters
    global trial_counters
    trial_counters=[0]*num_individuals
    print("Trial counters after initialization: \n",trial_counters)

    #Creating a list where we can store the best subset of features fetched from ABC algorithm
    global best_individual
    best_local_individual=list()
    best_local_fitness=list()
    #Splitting the dataset into training and validation set
    X_train, X_val, y_train, y_val=train_test_split(X, y, test_size=0.2, random_state=42)

    fitness_values=list()
    #Evaluating initial population
    for individual in population:
        fitness=fitness_function(individual, X_train, y_train, X_val, y_val, 1, 1)
        fitness_values.append(fitness)
    print("Fitness values after evaluating initial population: ",fitness_values)
    #Determining and memorizing the best fitness function value and the best solution from initial population
    f_best=max(fitness_values)
    f_best_index=fitness_values.index(f_best)
    best_individual=population[f_best_index]

    print("Best feature subset after generating initial population: ",best_individual)
    print("Best fitness value after generating initial population: ",f_best)
    print("Best fitness value lies at index: ",f_best_index)
    initial_population=population.copy()
    initial_fitness=fitness_values.copy()

    for generation in range(num_generations):

        #employed_bee_population=employed_bee_phase(population, X_train, y_train, X_val, y_val, generation, num_generations)
        #Employed bee phase
        population, fitness_values=employed_bee_phase(population, fitness_values, X_train, y_train, X_val, y_val, generation, num_generations)
        #Determining and memorizing the best fitness function value and the best solution from employee bee phase population
        f_best=max(fitness_values)
        f_best_index=fitness_values.index(f_best)
        best_individual=population[f_best_index]
        print("At generation: ",generation)
        print("Best feature subset generated after employee bee phase: ",best_individual)
        print("Best fitness value computed after employee bee phase: ",f_best)
        print("Best fitness value lies at index: ",f_best_index)

        phase1_population=population.copy()
        phase1_fitness=fitness_values.copy()

        #Onlooker bee phase
        population, fitness_values=onlooker_bee_phase(population, fitness_values, X_train, y_train, X_val, y_val, generation, num_generations)
        #Determining and memorizing the best fitness function value and the best solution from onlooker bee phase population
        f_best=max(fitness_values)
        f_best_index=fitness_values.index(f_best)
        best_individual=population[f_best_index]
        print("At generation: ",generation)
        print("Best feature subset generated after onlooker bee phase: ",best_individual)
        print("Best fitness value computed after onlooker bee phase: ",f_best)
        print("Best fitness value lies at index: ",f_best_index)

        phase2_population=population.copy()
        phase2_fitness=fitness_values.copy()

        best_local_fitness.append(f_best)
        best_local_individual.append(best_individual)

        #Scout bee phase
        '''
        In a given generation, we perform Scout bee phase for food source where trial is greater than limit.
        If we have more than one food source with trial greater than limit=5: -
        Case 1: trial_counters=[2, 18, 7, 6, 3]
        Here, food source 2, 3 and 4 have trial counter greater than limit. But only one of them can be passed to Scout phase.
        Thus, we select the food source with maximum trial_counter. In effect, we are taking the food source with maximum number of failures.

        Case 2: trial_counters=[6, 6, 6, 1, 3]
        Here, food source 1, 2 and 3 have trial counter lesser than limit and are equal. But, only one of them can be passed to Scout phase.
        Thus, we randomly select one of the three food sources for Scout phase.

        Alternatively, if we have trial_counters=[1, 2, 3, 4, 0]. Here, none of the 5 food sources have trial greater than limit. Thus, no food source
        goes to Scout phase.
        '''

        population, fitness_values=scout_bee_phase(population, fitness_values, X_train, y_train, X_val, y_val, generation, num_generations, limit)
        #Determining and memorizing the best fitness function value and the best solution from scout bee phase population
        f_best=max(fitness_values)
        f_best_index=fitness_values.index(f_best)
        best_individual=population[f_best_index]
        print("At generation: ",generation)
        print("Best feature subset generated after scout bee phase: ",best_individual)
        print("Best fitness value computed after scout bee phase: ",f_best)
        print("Best fitness value lies at index: ",f_best_index)

        phase3_population=population.copy()
        phase3_fitness=fitness_values.copy()

    #Comparing solution of last Scout bee phase and the best solution memorized after each Onlooker bee phase
    best_fitness_in_memory=max(best_local_fitness)
    index_best_fitness_in_memory=best_local_fitness.index(best_fitness_in_memory)
    best_individual_in_memory=best_local_individual[index_best_fitness_in_memory]

    print("Comparing solution of last Scout bee phase and the best solution memorized after each Onlooker bee phase")
    print("History of best fitness stored after each generation of Onlooker bee phase: ",best_local_fitness)
    print("Best fitness in memory: ",best_fitness_in_memory)
    print("Index of best fitness in memory: ",index_best_fitness_in_memory)
    print("History of best individual stored after each generation of Onlooker bee phase: ",best_local_individual)
    print("Best individual in memory: ",best_individual_in_memory)

    best_fitness_in_last_scout_bee_phase=max(fitness_values)
    #index_best_fitness_in_last_scout_bee_phase=fitness_values.index(best_fitness_in_last_scout_bee_phase)
    best_individual_in_last_scout_bee_phase=best_individual
    print("List of all fitness values in the last scout bee phase: ",fitness_values)
    print("Best fitness in last scout bee phase: ",best_fitness_in_last_scout_bee_phase)
    print("Best individual in scout bee phase: ",best_individual_in_last_scout_bee_phase)


    if best_fitness_in_memory>best_fitness_in_last_scout_bee_phase:
        print("Best fitness in memory is greater than best fitness in last scout bee phase")
        print("Best individual is: ",best_individual_in_memory)
        best_individual=best_individual_in_memory
    else:
        print("Best fitness in memory is smaller than best fitness in last scout bee phase")
        print("Best individual is: ",best_individual)

    return best_individual

In [10]:
#Reading the processed dataset
cic_df=pd.read_parquet("multiclass_training_data.parquet")

In [11]:
#Fetching the first 5 rows of the dataset
cic_df.head()

,Flow_Duration,Total_Fwd_Packets,Total_Backward_Packets,Fwd_Packets_Length_Total,Bwd_Packets_Length_Total,Fwd_Packet_Length_Max,Fwd_Packet_Length_Mean,Fwd_Packet_Length_Std,Bwd_Packet_Length_Max,Bwd_Packet_Length_Mean,...,Avg_Fwd_Segment_Size,Avg_Bwd_Segment_Size,Subflow_Fwd_Packets,Subflow_Fwd_Bytes,Subflow_Bwd_Packets,Subflow_Bwd_Bytes,Init_Fwd_Win_Bytes,Fwd_Act_Data_Packets,Fwd_Seg_Size_Min,attack_id
0,2999994.0,4.0,0.0,2064.0,0.0,516.0,44.000000,0.000000,0.0,0.00,...,44.000000,0.00,4.0,2064.0,0.0,0.0,8192.0,3.0,20.0,3
1,11487.0,3.0,4.0,326.0,129.0,326.0,108.666664,188.216187,112.0,32.25,...,108.666664,32.25,3.0,326.0,4.0,129.0,8192.0,1.0,20.0,1
2,9818.0,3.0,4.0,326.0,129.0,326.0,108.666664,188.216187,112.0,32.25,...,108.666664,32.25,3.0,326.0,4.0,129.0,8192.0,1.0,20.0,1
3,10138.0,3.0,4.0,326.0,129.0,326.0,108.666664,188.216187,112.0,32.25,...,108.666664,32.25,3.0,326.0,4.0,129.0,8192.0,1.0,20.0,1
4,109.0,3.0,1.0,53.0,0.0,53.0,17.666666,30.599564,0.0,0.00,...,17.666666,0.00,3.0,53.0,1.0,0.0,1080.0,1.0,20.0,0


In [12]:
#Computing shape of the dataset
cic_df.shape

(82184, 46)

In [13]:
#Dividing the dataset into Independent and Dependent features
X=cic_df.iloc[:, 0:-1]
y=cic_df.iloc[:, -1]

In [14]:
X

,Flow_Duration,Total_Fwd_Packets,Total_Backward_Packets,Fwd_Packets_Length_Total,Bwd_Packets_Length_Total,Fwd_Packet_Length_Max,Fwd_Packet_Length_Mean,Fwd_Packet_Length_Std,Bwd_Packet_Length_Max,Bwd_Packet_Length_Mean,...,Avg_Packet_Size,Avg_Fwd_Segment_Size,Avg_Bwd_Segment_Size,Subflow_Fwd_Packets,Subflow_Fwd_Bytes,Subflow_Bwd_Packets,Subflow_Bwd_Bytes,Init_Fwd_Win_Bytes,Fwd_Act_Data_Packets,Fwd_Seg_Size_Min
0,2999994.0,4.0,0.0,2064.0,0.0,516.0,44.000000,0.000000,0.0,0.00000,...,99.500000,44.000000,0.00000,4.0,2064.0,0.0,0.0,8192.0,3.0,20.0
1,11487.0,3.0,4.0,326.0,129.0,326.0,108.666664,188.216187,112.0,32.25000,...,65.000000,108.666664,32.25000,3.0,326.0,4.0,129.0,8192.0,1.0,20.0
2,9818.0,3.0,4.0,326.0,129.0,326.0,108.666664,188.216187,112.0,32.25000,...,65.000000,108.666664,32.25000,3.0,326.0,4.0,129.0,8192.0,1.0,20.0
3,10138.0,3.0,4.0,326.0,129.0,326.0,108.666664,188.216187,112.0,32.25000,...,65.000000,108.666664,32.25000,3.0,326.0,4.0,129.0,8192.0,1.0,20.0
4,109.0,3.0,1.0,53.0,0.0,53.0,17.666666,30.599564,0.0,0.00000,...,13.250000,17.666666,0.00000,3.0,53.0,1.0,0.0,1080.0,1.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82179,9749.0,3.0,4.0,326.0,129.0,326.0,108.666664,188.216187,112.0,32.25000,...,65.000000,108.666664,32.25000,3.0,326.0,4.0,129.0,8192.0,1.0,20.0
82180,343659.0,3.0,2.0,1912.0,232.0,640.0,86.909088,137.688019,976.0,121.13636,...,104.022728,86.909088,121.13636,3.0,1912.0,2.0,232.0,26883.0,1.0,32.0
82181,5401.0,2.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.00000,...,0.000000,0.000000,0.00000,2.0,0.0,0.0,0.0,32738.0,0.0,20.0
82182,412081.0,3.0,2.0,1912.0,232.0,640.0,83.130432,135.737488,976.0,121.13636,...,101.711113,83.130432,121.13636,3.0,1912.0,2.0,232.0,26883.0,1.0,32.0


In [15]:
y

,attack_id
0,3
1,1
2,1
3,1
4,0
...,...
82179,1
82180,2
82181,3
82182,2


In [16]:
#Standardizing distribution of independent features using Robust Scaler
robust_scaler=RobustScaler()
model=robust_scaler.fit(X)
scaled_data=model.transform(X)
print(scaled_data)
#Declaring global variables
trial_counters=list()
best_individual=list()

[[ 6.90830517  1.         -1.         ...  0.          2.
   0.        ]
 [-0.83683032  0.          1.         ...  0.          0.
   0.        ]
 [-0.84115577  0.          1.         ...  0.          0.
   0.        ]
 ...
 [-0.85260304 -1.         -1.         ...  1.31325237 -1.
   0.        ]
 [ 0.20136528  0.          0.         ...  1.          0.
   1.        ]
 [-0.07587805  9.         -1.         ...  0.          0.
   0.        ]]


In [17]:
best_features=abc_feature_selection(scaled_data, y)
print(f"Selected features: {best_features}")
print("Number of features: ",len(best_features))

abc_feature_selection method has started
num_features:  45
num_generations:  5
num_individuals:  22
limit:  5
initialize_population method has started
initialize_population method has ended
Initial population: 
 [[27, 5, 42, 41, 36, 8, 11], [34, 10, 25, 33, 5, 37, 30, 38, 4, 9, 40, 1, 39, 16, 26, 6, 32, 12, 20, 2, 44, 3, 28, 27, 23, 42], [13, 28, 2, 19, 20, 15, 5, 0, 21, 40, 32, 26, 10, 22, 8, 1, 12, 44, 7, 24, 14, 37, 31, 39, 23, 36, 6], [42, 9, 28, 40, 14, 3, 43, 26, 1, 34, 41, 35, 44, 18, 19, 33, 11, 7, 29, 21, 8, 13, 4, 16], [40, 9, 12, 4, 35, 38, 21, 29, 16, 22, 24, 34, 14, 6, 31, 2, 8, 25, 30, 43, 13, 17, 7, 28], [19, 36, 27, 40, 42, 14, 22, 13, 29, 21, 0, 18, 44, 10, 6, 30, 12, 8, 33, 35, 7, 5, 31, 28, 17, 9, 41, 4, 24, 34, 32, 2, 39, 38, 16, 11, 26, 23], [39, 15, 41, 44, 14, 26, 4, 1, 23, 21, 38, 16, 0, 27, 9, 36, 37, 11], [41, 9, 34, 30, 35, 44, 22, 4, 1, 8, 18, 21, 13], [9, 10, 37, 14, 28, 8, 33, 26, 35, 44, 29, 2, 38, 15, 17, 13, 12, 21, 23], [28, 30, 26, 16, 40, 9, 21, 3, 2